In [1]:
import numpy as np
import pandas as pd

In [2]:
n = int(input("Enter the total number of states in the DFA: "))
state = input("Enter the states in DFA: ").split()
symbol = input("Enter the symbols in the DFA: ").split()

In [3]:
# Mapping of the states to numeric values 

total_states = [i for i in range(n)]
w = []
for i in range(len(state)):
    w.append((i, state[i]))
mapping = dict(w)

In [4]:
Temp = input("Enter the final states in DFA: ").split()

In [5]:
# Mapping for Final States

final = []

for i in Temp:
    for k in mapping:
        if mapping[k] == i:
            final.append(k)
            break
final

[1, 2, 5]

In [10]:
# Transaction table of the DFA

transaction_table = np.zeros(shape=(n,len(symbol)),dtype='int32')
for_display = []
for i in range(0, n):
    t = []
    for k in range(len(symbol)):
        temp = input("Enter the state whose transition is from " +state[i]+ " state for symbol " +symbol[k])
        t.append(temp)
        for j in mapping:
            if mapping[j] == temp:
                temp = j
                break
        transaction_table[i][k] = temp
    for_display.append(t)
for_display

[['Q1', 'Q2'],
 ['Q0', 'Q3'],
 ['Q4', 'Q5'],
 ['Q4', 'Q5'],
 ['Q4', 'Q5'],
 ['Q5', 'Q5']]

In [11]:
print("Shape: " + str(transaction_table.shape))
transaction_table

Shape: (6, 2)


array([[1, 2],
       [0, 3],
       [4, 5],
       [4, 5],
       [4, 5],
       [5, 5]])

In [12]:
# Removing in-accessible states 

y = set([0])
previous_length = len(y)

while True:
    m = set()
    for i in y:
        t = set(transaction_table[i])
        m = m.union(t)
    y = y.union(m)
    if previous_length == len(y):
        break
    else:
        previous_length = len(y)

print("Valid States in DFA are - ")
print(y, end='\n\n')
unreachable_state = set(total_states) - y
print("Unreachable state are - ")
print(unreachable_state)

Valid States in DFA are - 
{0, 1, 2, 3, 4, 5}

Unreachable state are - 
set()


In [13]:
# Removing Unreachable State from the entire transactional data

if len(unreachable_state) == 0:
    print(transaction_table)
else:
    n -= len(unreachable_state)
    for i in unreachable_state:
        transaction_table = np.delete(transaction_table, i, 0)
        total_states.remove(i)
        del for_display[i]
        del state[i]

[[1 2]
 [0 3]
 [4 5]
 [4 5]
 [4 5]
 [5 5]]


In [14]:
print("Transaction Table is ")
table = pd.DataFrame(for_display,columns=symbol,index=state)
table

Transaction Table is 


,0,1
Q0,Q1,Q2
Q1,Q0,Q3
Q2,Q4,Q5
Q3,Q4,Q5
Q4,Q4,Q5
Q5,Q5,Q5


In [15]:
# inital Iteration

temp = np.zeros(shape=(n, n))

for i in range(0, n):
    for j in range(0, i):
        if (i in final and j not in final) or (i not in final and j in final):
            temp[i][j] = 1
    for j in range(i, n):
        temp[i][j] = -1
temp

array([[-1., -1., -1., -1., -1., -1.],
       [ 1., -1., -1., -1., -1., -1.],
       [ 1.,  0., -1., -1., -1., -1.],
       [ 0.,  1.,  1., -1., -1., -1.],
       [ 0.,  1.,  1.,  0., -1., -1.],
       [ 1.,  0.,  0.,  1.,  1., -1.]])

In [16]:
count = (n * (n-1)) // 2
prev_count = count

while True:
    for i in range(1, n):
        for j in range(0, i):
            for k in range(len(symbol)): 
                if temp[i][j] == 1:
                    continue             
                elif temp[int(transaction_table[i][k])][int(transaction_table[j][k])] == 1 or \
                        temp[int(transaction_table[j][k])][int(transaction_table[i][k])] == 1:
                    temp[i][j] = 1
                    count -= 1

    if count == prev_count:
        break
    else:
        prev_count = count

In [17]:
print("Final Table")
temp

Final Table


array([[-1., -1., -1., -1., -1., -1.],
       [ 1., -1., -1., -1., -1., -1.],
       [ 1.,  1., -1., -1., -1., -1.],
       [ 1.,  1.,  1., -1., -1., -1.],
       [ 1.,  1.,  1.,  0., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1., -1.]])

In [18]:
# finding the states which can be combined

possible_combine = []
for i in range(n):
    for j in range(0, i):
        if temp[i][j] == 0:
            possible_combine.append([i, j])

possible_combine

[[4, 3]]

In [19]:
# transitive combination possible

temp = []

for i in possible_combine:
    for j in possible_combine:
        if i[1] == j[0] and [i[0], j[1]] in possible_combine:
            temp.append([i[0], i[1], j[1]])
            break
    else:
        temp.append(i)
temp

[[4, 3]]

In [20]:
final_combination = []
x = []
for i in range(len(temp)):
    c = 0
    for j in range(i + 1, len(temp)):
        if len(set(temp[i]) - set(temp[j])) == 0:
            c += 1
    if c == 0:
        final_combination.append(temp[i])
        x.extend(temp[i])
final_combination

[[4, 3]]

In [21]:
total_states

[0, 1, 2, 3, 4, 5]

In [22]:
final_states = final_combination.copy()
x = list(set(total_states) - set(x))
for i in x:
    final_states.append([i])

final_states

[[4, 3], [0], [1], [2], [5]]

In [23]:
length = len(final_states)

print("Total Number of states in the Minimized DFA is: " + str(length))

Total Number of states in the Minimized DFA is: 5


In [24]:
# final Transitions

final_transtions = []

for i in final_states:
    t = i[0]
    r = []
    for k in range(len(symbol)):
        e = transaction_table[t][k]
        for j in final_states:
            if e in j:
                str = ''
                for k in j:
                    str += mapping[k]
                break
        r.append(str)
    final_transtions.append(r)

In [25]:
# indexes generation

index = []
for i in range(len(final_states)):
    str = ''
    for j in range(len(final_states[i])):
        str += mapping[final_states[i][j]]
    index.append(str)

In [26]:
print("The Minimized Transition Table is : ")

df = pd.DataFrame(final_transtions,columns=symbol,index=index)
df

The Minimized Transition Table is : 


,0,1
Q4Q3,Q4Q3,Q5
Q0,Q1,Q2
Q1,Q0,Q4Q3
Q2,Q4Q3,Q5
Q5,Q5,Q5


In [32]:
def dfa_to_regex(dfa):
    # Step 1: Create a matrix representing the system of equations
    n = len(dfa.states)
    matrix = [[None for j in range(n)] for i in range(n)]
    for i, state_i in enumerate(dfa.states):
        for j, state_j in enumerate(dfa.states):
            if state_j in dfa.transitions[state_i]:
                matrix[i][j] = dfa.transitions[state_i][state_j]
        if i == j:
            matrix[i][j] = "(1)" # Add a 1 in the diagonal

    # Step 2: Solve the system of equations using Arden's Theorem
    for k in range(n):
        for i in range(n):
            if matrix[i][k] is not None:
                for j in range(n):
                    if matrix[k][j] is not None:
                        if matrix[i][j] is None:
                            matrix[i][j] = matrix[i][k] + "(" + matrix[k][k] + ")*" + matrix[k][j]
                        else:
                            matrix[i][j] = matrix[i][j] + "+" + matrix[i][k] + "(" + matrix[k][k] + ")*" + matrix[k][j]

    # Step 3: Extract the regular expression for the start and accept state
    start_state = dfa.start_state
    accept_state = dfa.accept_states[0]
    return matrix[start_state][accept_state]

In [29]:
class DFA:
    def __init__(self, states, transitions, start_state, accept_states):
        self.states = states
        self.transitions = transitions
        self.start_state = start_state
        self.accept_states = accept_states





states = [0, 1, 2, 3, 4, 5]
transitions = {
    0: {'a': 1, 'b': 2},
    1: {'a': 3, 'b': 4},
    2: {'a': 4, 'b': 3},
    3: {'a': 5, 'b': 5},
    4: {'a': 5, 'b': 5},
    5: {'a': 5, 'b': 5}
}
start_state = 0
accept_states = {1, 2, 5}

dfa = DFA(states, transitions, start_state, accept_states)

In [33]:
dfa_to_regex(dfa)

TypeError: 'set' object is not subscriptable